In [84]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import lightgbm as lgb
import catboost as cb
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features

import optuna

from common import EP
from dfdb import DFDB

import types

In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [43]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_colwidth', -1)

In [3]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [4]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [5]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [6]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [7]:
mytrial = []
# mytrial = list(pd.read_pickle('trial/catboost.pkl').T.to_dict().values())
# df_trial = pd.DataFrame(mytrial)
# len(mytrial)

In [9]:
columns = ['q25_roll_std_100',
 'spkt_welch_densitycoeff_2',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'abs_q75_7',
 'abs_q25_5',
 'spkt_welch_density__coeff_3',
 'abs_q75_6',
 'q01_roll_std_1000',
 'Hilbert_mean_6',
 'abs_q95_2',
 'FFT_Mag_75q0',
 'abs_q01_5',
 '5000std_median_']

In [10]:
len(columns)

13

In [85]:
db = DFDB('../trial/mlp.pkl', auto_commit=False)

In [106]:
param={
    'algorithm': {
        'cls': 'MLPRegressor',
        'fit': {
        },
        'init': {
            'hidden_layer_sizes':(64, 16), 
            'activation':'relu', 
            'solver':'sgd', 
            'alpha':0.001, 
            'learning_rate':'adaptive', 
            'learning_rate_init':0.001, 
            'beta_1':0.9, 
            'beta_2':0.999,
            'batch_size':128, 
            'max_iter':100, 
            'power_t':0.5, 
            
            'shuffle':True, 
            'random_state':42, 
            'tol':0.000001, 
            'warm_start':False, 
            'momentum':0.9, 
            'nesterovs_momentum':False, 
            'validation_fraction':0, 
            'epsilon':1e-08, 
            'early_stopping':False, 
            'n_iter_no_change':20
        }
    },
    'columns': columns,

    'kfold': {
        'n_splits': 3,
        'random_state': 1985,
        'shuffle': True,
        'type': 'stratified'
    },
    'scaler': {
        'cls': 'StandardScaler',
        'init':{}
    }
}

In [107]:
# run one try
# mytrial = []
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = df_test, trial=mytrial)

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.



In [108]:
df_trial = pd.DataFrame(mytrial)
df_trial['algorithm-init']=df_trial['param'].apply(lambda x: x['algorithm']['init'])
df_trial['kfold']=df_trial['param'].apply(lambda x: x['kfold'])

In [109]:
#[df_trial['mae_diff']<.05].sort_values(by=['val_mae']) | (df_trial['remark']=='tune feats selected by group ')
df_trial[['datetime','nfeatures','kfold', 'algorithm-init', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,kfold,algorithm-init,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
0,2019-05-25 21:40:39.050681,13,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}","{'hidden_layer_sizes': (64, 16), 'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'beta_1': 0.9, 'beta_2': 0.999, 'batch_size': 32, 'max_iter': 100, 'power_t': 0.5, 'shuffle': True, 'random_state': 42, 'tol': 1e-06, 'warm_start': False, 'momentum': 0.9, 'nesterovs_momentum': False, 'validation_fraction': 0, 'epsilon': 1e-08, 'early_stopping': False, 'n_iter_no_change': 20}",1.878430,0.011985,2.080568,0.503640,0.202138
1,2019-05-25 22:06:40.926665,13,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}","{'hidden_layer_sizes': (64, 16), 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'beta_1': 0.9, 'beta_2': 0.999, 'batch_size': 32, 'max_iter': 100, 'power_t': 0.5, 'shuffle': True, 'random_state': 42, 'tol': 1e-06, 'warm_start': False, 'momentum': 0.9, 'nesterovs_momentum': False, 'validation_fraction': 0, 'epsilon': 1e-08, 'early_stopping': False, 'n_iter_no_change': 20}",1.884322,0.010035,2.079307,0.535156,0.194985
2,2019-05-25 22:09:42.533016,13,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}","{'hidden_layer_sizes': (32, 16), 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'beta_1': 0.9, 'beta_2': 0.999, 'batch_size': 128, 'max_iter': 100, 'power_t': 0.5, 'shuffle': True, 'random_state': 42, 'tol': 1e-06, 'warm_start': False, 'momentum': 0.9, 'nesterovs_momentum': False, 'validation_fraction': 0, 'epsilon': 1e-08, 'early_stopping': False, 'n_iter_no_change': 20}",1.954309,0.010999,2.089110,0.499385,0.134801
3,2019-05-25 22:13:10.030178,13,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}","{'hidden_layer_sizes': (32, 16), 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'beta_1': 0.9, 'beta_2': 0.999, 'batch_size': 128, 'max_iter': 100, 'power_t': 0.5, 'shuffle': True, 'random_state': 42, 'tol': 1e-06, 'warm_start': False, 'momentum': 0.9, 'nesterovs_momentum': False, 'validation_fraction': 0, 'epsilon': 1e-08, 'early_stopping': False, 'n_iter_no_change': 20}",1.948725,0.002606,2.059295,0.015746,0.110570
4,2019-05-25 22:14:36.531081,13,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}","{'hidden_layer_sizes': (32, 8), 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'beta_1': 0.9, 'beta_2': 0.999, 'batch_size': 128, 'max_iter': 100, 'power_t': 0.5, 'shuffle': True, 'random_state': 42, 'tol': 1e-06, 'warm_start': False, 'momentum': 0.9, 'nesterovs_momentum': False, 'validation_fraction': 0, 'epsilon': 1e-08, 'early_stopping': False, 'n_iter_no_change': 20}",1.941612,0.001733,2.076839,0.014184,0.135227
5,2019-05-25 22:23:20.167106,13,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}","{'hidden_layer_sizes': (64, 16), 'activation': 'relu', 'solver': 'sgd', 'alpha': 1e-05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'beta_1': 0.9, 'beta_2': 0.999, 'batch_size': 128, 'max_iter': 100, 'power_t': 0.5, 'shuffle': True, 'random_state': 42, 'tol': 1e-06, 'warm_start': False, 'momentum': 0.9, 'nesterovs_momentum': False, 'validation_fraction': 0, 'epsilon': 1e-08, 'early_stopping': False, 'n_iter_no_change': 20}",1.906227,0.002796,2.047368,0.013625,0.141141
6,2019-05-25 22:31:34.893667,13,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}","{'hidden_layer_sizes': (64, 16), 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'beta_1': 0.9, 'beta_2': 0.999, 'batch_size': 128, 'max_iter': 100, 'power_t': 0.5, 's

In [110]:
len(mytrial)

20

In [111]:
for trial_i in mytrial:
    db.insert(trial_i)

In [112]:
db.commit()